# Exemplo Prática Ciência de Dados

### A Administração de Seguridade Social dos Estados Unidos (SSA) disponibilizou dados sobre a frequência de nomes de bebês de 1880 até o presente. Neste exemplo, vamos usar de 

#### Há muitas coisas que você pode querer fazer com o conjunto de dados:
#### • Visualize a proporção de bebês com um nome específico (o seu próprio ou outro nome) ao longo do tempo
#### • Determinar a classificação relativa de um nome
#### • Determine os nomes mais populares em cada ano ou os nomes cuja popularidade avançou ou declinou mais
#### • Analisar tendências em nomes: vogais, consoantes, comprimento, diversidade geral, mudanças no ortografia, primeiras e últimas letras
#### • Analisar fontes externas de tendências: nomes bíblicos, celebridades, dados demográficos mudanças

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
url1 = "https://raw.githubusercontent.com/rlrocha90/IAeML-Treinamento/main/names1.csv"
url2 = "https://raw.githubusercontent.com/rlrocha90/IAeML-Treinamento/main/names2.csv"
url3 = "https://raw.githubusercontent.com/rlrocha90/IAeML-Treinamento/main/names3.csv"
url4 = "https://raw.githubusercontent.com/rlrocha90/IAeML-Treinamento/main/names4.csv"
names1 = pd.read_csv(url1)
names2 = pd.read_csv(url2) 
names3 = pd.read_csv(url3)
names4 = pd.read_csv(url4)
names = []
names.append(names1)
names.append(names2)
names.append(names3)
names.append(names4)
names = pd.concat(names, axis=0, ignore_index=True)

In [ ]:
names.head()

#### Com esses dados em mãos, já podemos começar a agregar os dados no nível de ano e sexo usando groupby ou pivot_table
#### PivotTable para agregar (somar) os nascimentos por ano e sexo

In [ ]:
total_births = names.pivot_table("births", index="year", columns="sex", aggfunc=sum)
total_births.plot(title='Total de Nascimentos por sexo e ano')
plt.show()

#### Em seguida, vamos inserir um suporte de coluna com a fração de bebês com cada nome em relação ao número
#### total de nascimentos. Um valor prop de 0,02 indicaria que 2 em cada 100 bebês receberam um nome específico.
#### Assim, agrupamos os dados por ano e sexo e adicionamos a nova coluna a cada grupo:

In [ ]:
def add_prop(group):
    group["prop"] = group["births"] / group["births"].sum()
    return group

names = names.groupby(["year", "sex"]).apply(add_prop)

#### O conjunto de dados completo resultante agora tem as seguintes colunas:

In [ ]:
print(names)

#### Ao realizar uma operação de grupo como esta, geralmente é valioso fazer uma verificação de sanidade,
#### como verificar se a coluna prop soma 1 em todos os grupos:

In [ ]:
print(names.groupby(["year", "sex"])["prop"].sum())

#### Agora que isso está feito, vou extrair um subconjunto dos dados para facilitar uma análise mais aprofundada:
#### os 1.000 principais nomes para cada combinação de sexo/ano. Esta é mais uma operação de grupo:

In [ ]:
def get_top1000(group):
    return group.sort_values("births", ascending=False)[:1000]

grouped = names.groupby(["year", "sex"])
top1000 = grouped.apply(get_top1000)
top1000 = top1000.reset_index(drop=True)

#### O conjunto de dados resultante agora é um pouco menor:

In [ ]:
print(top1000.head())

### Analisando tendências de nomes
#### Com o conjunto de dados completo e o conjunto de dados Top 1.000 em mãos, podemos começar
#### a analisar várias tendências de nomes de interesse. Dividir os 1.000 principais nomes nas partes de menino e menina é fácil de fazer primeiro:

In [ ]:
boys = top1000[top1000["sex"] == "M"]
girls = top1000[top1000["sex"] == "F"]

#### Séries temporais simples, como o número de Johns ou Marys para cada ano, podem ser plotadas, mas requerem
#### um pouco de atenção para serem mais úteis. Vamos formar uma tabela dinâmica do número total de nascimentos por ano e nome:

In [ ]:
total_births = top1000.pivot_table("births", index="year", columns="name", aggfunc=sum)

#### Agora, isso pode ser plotado para um punhado de nomes com o método de plotagem do DataFrame

In [ ]:
print("---- informações do Total de Nascimento ----")
print(total_births.info())
subset = total_births[["Anna", "Sarah", "Minnie", "Marilyn"]]
subset.plot(subplots=True, figsize=(12, 10), title="Número de Nascimentos por ano")
plt.show()

#### Ao olhar para isso, você pode concluir que esses nomes cresceram em desuso com a população americana. Mas a história é realmente mais complicada do que isso, como será explorado na próxima seção.

### Medindo o aumento na diversidade de nomes
#### Uma explicação para a diminuição das parcelas é que menos pais estão escolhendo nomes comuns para seus filhos.
#### Essa hipótese pode ser explorada e confirmada nos dados. Uma medida é a proporção de nascimentos representados pelos 1.000 nomes mais populares, que eu agrego e gravo por ano e sexo

In [ ]:
table = top1000.pivot_table("prop", index="year", columns="sex", aggfunc=sum)
table.plot(title="Soma de table1000.prop por ano e sexo", yticks=np.linspace(0, 1.2, 13))
plt.show()

#### Você pode ver que, de fato, parece estar aumentando a diversidade de nomes (diminuindo a proporção total no top 1.000). Outra métrica interessante é o número de nomes distintos, em ordem de popularidade, do mais alto para o mais baixo, nos 50% melhores nascimentos. Este número é um pouco mais complicado de calcular. Vamos considerar apenas os nomes de meninos de 2010:

In [ ]:
df = boys[boys["year"] == 2010]
print(df)

#### Depois de classificar prop em ordem decrescente, queremos saber quantos dos nomes mais populares são necessários para chegar a 50%. Você poderia escrever um loop for para fazer isso, mas uma maneira NumPy vetorizada é um pouco mais inteligente. Pegar a soma cumulativa, cumsum, de prop e então chamar o método searchsorted retorna a posição na soma cumulativa na qual 0.5 precisaria ser inserido para mantê-la em ordem:

In [ ]:
prop_cumsum = df["prop"].sort_values(ascending=False).cumsum()
print(prop_cumsum[0:10])
plt.hist(prop_cumsum)
plt.show()
prop_cumsum.plot.density()
plt.show()
print(prop_cumsum.searchsorted(0.5))

#### Como as matrizes são indexadas a zero, adicionar 1 a esse resultado resulta em 117. Em contraste, em 1900 esse número era muito menor:

In [ ]:
df = boys[boys.year == 1900]
in1900 = df.sort_values("prop", ascending=False).prop.cumsum()
print(in1900.searchsorted(0.5) + 1)

#### Agora você pode aplicar esta operação a cada combinação de ano/sexo, agrupar por esses campos e aplicar uma função que retorna a contagem para cada grupo:

In [ ]:
def get_quantile_count(group, q=0.5):
    group = group.sort_values("prop", ascending=False)
    return group.prop.cumsum().searchsorted(q) + 1

diversity = top1000.groupby(["year", "sex"]).apply(get_quantile_count)
diversity = diversity.unstack()

#### Essa diversidade de DataFrame resultante agora tem duas séries temporais, uma para cada sexo, indexadas por ano.

In [ ]:
print(diversity.head())
diversity.plot(title="Número de nomes populares no top 50%")
plt.show()

#### Como você pode ver, os nomes das meninas sempre foram mais diversos do que os nomes dos meninos, e eles só se tornaram mais ao longo do tempo. Uma análise mais aprofundada do que exatamente está impulsionando a diversidade, como o aumento de grafias alternativas, é deixada para o leitor.

### A revolução da “última letra”
#### Em 2007, a pesquisadora de nomes de bebês Laura Wattenberg apontou em seu site que a distribuição de nomes de meninos por letra final mudou significativamente nos últimos 100 anos. Para ver isso, primeiro agregamos todos os nascimentos no conjunto de dados completo por ano, sexo e letra final:

In [ ]:
def get_last_letter(x):
    return x[-1]

last_letters = names["name"].map(get_last_letter)
last_letters.name = "last_letter"

table = names.pivot_table("births", index=last_letters, columns=["sex", "year"], aggfunc=sum)

#### Em seguida, selecionamos três anos representativos abrangendo o histórico e imprimimos as primeiras linhas:

In [ ]:
subtable = table.reindex(columns=[1910, 1960, 2010, 2020], level="year")
print(subtable.head())

#### Em seguida, normalize a tabela pelo total de nascimentos para calcular uma nova tabela contendo a proporção do total de nascimentos para cada sexo terminando em cada letra:

In [ ]:
print(subtable.sum())
letter_prop = subtable / subtable.sum()
print(letter_prop)

#### Com as proporções das letras agora em mãos, podemos fazer gráficos de barras para cada sexo divididos por ano

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(10, 8))
letter_prop["M"].plot(kind="bar", rot=0, ax=axes[0], title="Masculino")
letter_prop["F"].plot(kind="bar", rot=0, ax=axes[1], title="Feminino",legend=False)
plt.subplots_adjust(hspace=0.25)
plt.show()

#### Como você pode ver, os nomes de meninos terminados em n tiveram um crescimento significativo desde a década de 1960. Voltando à tabela completa criada anteriormente, normalizo novamente por ano e sexo e seleciono um subconjunto de letras para os nomes dos meninos, finalmente transpondo para tornar cada coluna uma série temporal:

In [ ]:
letter_prop = table / table.sum()

dny_ts = letter_prop.loc[["t", "n", "y"], "M"].T
print(dny_ts.head())

#### Com este DataFrame de séries temporais em mãos, posso fazer um gráfico das tendências ao longo do tempo novamente com seu método de plotagem

In [ ]:
dny_ts.plot()
plt.show()

### Nomes de meninos que se tornaram nomes de meninas (e vice-versa)
#### Outra tendência divertida é olhar para nomes de meninos que eram mais populares com um sexo no início da amostra, mas que “mudaram de sexo” no presente. Um exemplo é o nome Lesley ou Leslie. Voltando ao top1000

In [ ]:
all_names = pd.Series(top1000["name"].unique())
lesley_like = all_names[all_names.str.contains("Lesl")]
print(lesley_like)

#### A partir daí, podemos filtrar apenas esses nomes e somar nascimentos agrupados por nome para ver as frequências relativas:

In [ ]:
filtered = top1000[top1000["name"].isin(lesley_like)]
print(filtered.groupby("name")["births"].sum())

#### Em seguida, vamos agregar por sexo e ano e normalizar dentro do ano:

In [ ]:
table = filtered.pivot_table("births", index="year", columns="sex", aggfunc="sum")
table = table.div(table.sum(axis="columns"), axis="index")
table.tail()

#### Por fim, agora é possível fazer um gráfico da divisão por sexo ao longo do tempo

In [ ]:
table.plot(style={"M": "k-", "F": "k--"})
plt.show()